In [1]:
import pandas as pd
import datetime
from datetime import datetime, timedelta
import os
import polars as pl
import gc
import numpy as np
from tqdm import tqdm
import s3fs
import boto3
from io import BytesIO as bo

In [2]:
#variables -
this_day = datetime.today()
### FOR TESTING - REMOVE LATER ### 
this_day = this_day - timedelta(days=7)
####
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)

CUR_PROC_WK = monday.strftime("%Y%m%d")

PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)

PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")

In [3]:
# Grabbing IW Release File 
s3 = s3fs.S3FileSystem()
bucket_list = s3.listdir(f'vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/inbound/')
for file in bucket_list:
    if (file['Key'].__contains__(f"IRWD_RELEASE_WKLY_{CUR_PROC_WK}")):
        release_file = file['Key']

# Reading Release File -
s3_client = boto3.client('s3')
bucket = release_file.split('/')[0]
file_key = release_file.split('/',1)[1]

adm = pd.read_csv(f's3://{bucket}/{file_key}',sep='|')
adm1 = adm[CUR_PROC_WK].tolist()

for item in adm1:
    if item.startswith('P_OUT_110_FORMULARY_GROUP_REDUCED_'):
        FRM_TIMESTAMP_1 = item.split('_')[-1].split('.')[0]
    elif item.startswith('P_OUT_110_FORMULARY_GROUP_'):
        FRM_TIMESTAMP_2 = item.split('_')[-1].split('.')[0]

In [4]:
#LIBNAMES
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
prewk = f'PYADM/raw/{PRE_PROC_WK}/dataframes/'
ptk = f'PYADM/weekly/archive/{CUR_WK}/plantrak/'

### Start

In [8]:
input_file = f's3://{bucket}/{raw_path}P_OUT_110_FORMULARY_GROUP_REDUCED_{FRM_TIMESTAMP_1}.TXT'

FORMULARY = pd.read_csv(
    input_file,delimiter='|',
    dtype = {0:str}
)
FORMULARY = pl.from_pandas(FORMULARY)
FORMULARY = FORMULARY.unique(keep = 'first')

In [6]:
group_type_2_mapping_dict = {
    "Cash": "Cash",
    "Com": "Commercial",
    "FFS": "Medicaid",
    "HIX": "Commercial",
    "Mgd Medicaid": "Medicaid",
    "Part D": "Medicare Part D",
    "Voucher": "Other"
} # add clause for "" data or none data as well if needed

#mapping_dict = {**mapping_dict, **df["GROUP_TYPE"].unique().to_dict()} before the mapping operation. 
#This will add all unique values in GROUP_TYPE to the dictionary with the same value.

FORMULARY0 = FORMULARY.with_columns(
    pl.col("GROUP_TYPE").replace(group_type_2_mapping_dict).alias("GROUP_TYPE2")
)

In [7]:
brand_to_pfam_cd_mapping_dict = {
    "AMZ": "LUB",
    "APNL": "ALP",
    "CAN": "CAN",
    "DEL": "DEL",
    "DUZ": "DUZ",
    "LIN": "LIN",
    "TRU": "TRU",
    "ULC": "ULT",
    "VZ": "VIB",
    "XFN": "XIF",
    "ZUR": "ZUR",
    "LUB": "LUB",
    "MOT": "MOT",
    "IBR": "IRL",  # Ibsrela
    "ZEL": "ZEL",  # Zelnorm
    "CON": "LAC",  # Constulose
    "ENU": "LAC",  # Enulose
    "GEC": "LAC",  # Genralac
    "KRI": "LAC",  # Kristalose
    "LAC": "LAC",  # Lactulose
    None: "XXX",
    "": "XXX"  # For blank rows
}
FORMULARY1 = FORMULARY0.with_columns(
    pl.col("BRAND").replace(brand_to_pfam_cd_mapping_dict).alias("PFAM_CD")
)

In [8]:
std_sec_def = pl.read_parquet(f's3://{bucket}/PYADM/reference/std_sec_def.parquet',columns = ['PFAM_CD', 'PFAM_NAME', 'MKT_CD', 'MKT_NAME'])

#since you are only keeping 4 columns you need to drop dups
std_sec_def = std_sec_def.unique(subset=['PFAM_CD', 'PFAM_NAME', 'MKT_CD', 'MKT_NAME'])

In [9]:
FORMULARY2 = FORMULARY1.join(std_sec_def,on='PFAM_CD',how='inner')

In [10]:
# Uncomment for QC
# frequency_table = FORMULARY2.to_pandas().groupby(['FORMULARY_GROUP_STATUS']).size().reset_index(name='FREQUENCY')
# frequency_table

In [11]:
# frequency_table = FORMULARY2.to_pandas().groupby(['BRAND']).size().reset_index(name='FREQUENCY')
# frequency_table

In [12]:
# FORMULARY2.write_parquet(ptk+'\\FORMULARY.parquet',compression='zstd',compression_level=10,use_pyarrow=True)
FORMULARY2.to_pandas().to_parquet(f's3://{bucket}/{ptk}FORMULARY.parquet', compression='snappy')
# #Export Done!

## Full Formulary For Steve

In [13]:
#input_file = raw_path + 'P_OUT_110_FORMULARY_GROUP_' + str(FRM_TIMESTAMP_2) + ".TXT"
input_file = f's3://{bucket}/{raw_path}P_OUT_110_FORMULARY_GROUP_{FRM_TIMESTAMP_2}.TXT'
FORMULARY = pd.read_csv(
    input_file,delimiter='|',
    dtype = {0:str,1:str,4:str,8:str},
    parse_dates=[10]
)
FORMULARY['DATA_EFFECITVE_DATE'] = FORMULARY['DATA_EFFECITVE_DATE'].dt.date


In [14]:
dedup1 = ['IMS_PLAN_ID', 'IRWD_FGN_NAME', 'BRAND', 'GROUP_TYPE', 'FORMULARY_GROUP_STATUS','PARENT_NM','STATUS']

b = FORMULARY.shape[0]
FORMULARY.drop_duplicates(subset=dedup1, keep='first', inplace=True, ignore_index=False)
a = FORMULARY.shape[0]
print("Records Dropped : ",b-a)


Records Dropped :  0


In [15]:
sort_list1= ['IMS_PLAN_ID', 'IRWD_FGN_NAME', 'BRAND', 'GROUP_TYPE', 'FORMULARY_GROUP_STATUS', 'STATUS','LIVES']
sort_type1= [True]*6 + [False]
FORMULARY.sort_values(by=sort_list1,ascending=sort_type1,inplace=True,ignore_index=True)

In [16]:
dedup2 = ['IMS_PLAN_ID', 'IRWD_FGN_NAME' ,'BRAND', 'GROUP_TYPE' ,'FORMULARY_GROUP_STATUS', 'STATUS']
b = FORMULARY.shape[0]
FORMULARY.drop_duplicates(subset=dedup2, keep='first', inplace=True, ignore_index=False)
a = FORMULARY.shape[0]
print("Records Dropped : ",b-a)

Records Dropped :  0


In [17]:
FORMULARY = pl.from_pandas(FORMULARY)
FORMULARY0 = FORMULARY.with_columns(
    pl.col("GROUP_TYPE").replace(group_type_2_mapping_dict).alias("GROUP_TYPE2")
)
FORMULARY1 = FORMULARY0.with_columns(
    pl.col("BRAND").replace(brand_to_pfam_cd_mapping_dict).alias("PFAM_CD")
)

FORMULARY2 = FORMULARY1.join(std_sec_def,on='PFAM_CD',how='inner')
FORMULARY2 = FORMULARY2.sort(by='PFAM_CD')

In [18]:
#FORMULARY2.write_parquet(ptk+'\\FORMULARY_orig.parquet',compression='zstd',compression_level=10,use_pyarrow=True)
FORMULARY2.to_pandas().to_parquet(f's3://{bucket}/{ptk}FORMULARY_orig.parquet',compression='snappy')
#Export Done!